In [20]:
import numpy as np
import os
import time

def read_benchmark(file_path):
    """Lecture et parsing des fichiers benchmark"""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Lecture des deux premières valeurs : Taille de l'univers et nombre de sous-ensembles attendus
    universe_size, num_subsets_expected = map(int, lines[0].split())

    # Chargement des sous-ensembles
    subsets = [set(map(int, line.split())) for line in lines[1:]]

    num_subsets_loaded = len(subsets)

    if num_subsets_loaded != num_subsets_expected:
        print(f"⚠️ Avertissement : {file_path} indique {num_subsets_expected} sous-ensembles,"
              f" mais seulement {num_subsets_loaded} ont été chargés !")

    return universe_size, num_subsets_loaded, subsets

def fitness_func(solution, subsets, universe_size):
    """Évalue une solution en mesurant le nombre d'éléments couverts"""
    covered_elements = set()
    
    for idx, selected in enumerate(solution):
        if selected:
            if idx >= len(subsets):  # Vérification pour éviter IndexError
                print(f"⚠️ Erreur: idx={idx} hors limites, taille subsets={len(subsets)}")
                continue
            covered_elements.update(subsets[idx])
    
    num_covered = len(covered_elements)
    score = num_covered / universe_size  # Normalisation
    
    return num_covered, score  # Retourne les deux valeurs

def pso_mcp(subsets, universe_size, num_subsets, num_particles=75, num_iterations=100, 
            c1=2, c2=1, w=1, vmax=20, early_stop=1000):
    """Optimisation du Maximum Covering Problem avec Particle Swarm Optimization"""
    
    np.random.seed(42)  # Fixer la seed pour reproductibilité
    particles = np.random.choice([0, 1], size=(num_particles, num_subsets))  # Population aléatoire
    velocities = np.zeros((num_particles, num_subsets))
    personal_best = particles.copy()
    personal_best_scores = [fitness_func(p, subsets, universe_size) for p in particles]

    # Initialisation du meilleur global
    global_best = max(personal_best, key=lambda p: fitness_func(p, subsets, universe_size)[1])
    global_best_covered, global_best_score = fitness_func(global_best, subsets, universe_size)

    best_scores = []  # Historique des scores
    no_improve_counter = 0  # Pour l'early stopping
    
    start_time = time.time()  # Mesurer le temps d'exécution

    for iteration in range(num_iterations):
        for i in range(num_particles):
            r1, r2 = np.random.rand(num_subsets), np.random.rand(num_subsets)
            
            # Mise à jour de la vitesse
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (personal_best[i] - particles[i]) +
                             c2 * r2 * (global_best - particles[i]))
            
            velocities[i] = np.clip(velocities[i], -vmax, vmax)  # Appliquer Vmax
            
            # Mise à jour des positions
            particles[i] = (np.random.rand(num_subsets) < 1 / (1 + np.exp(-velocities[i]))).astype(int)

            # Évaluation
            num_covered, score = fitness_func(particles[i], subsets, universe_size)

            # Mise à jour du meilleur personnel
            if score > personal_best_scores[i][1]:  # Comparer le score normalisé
                personal_best[i] = particles[i].copy()
                personal_best_scores[i] = (num_covered, score)

            # Mise à jour du meilleur global
            if score > global_best_score:
                global_best = particles[i].copy()
                global_best_covered, global_best_score = num_covered, score
                no_improve_counter = 0  # Réinitialisation du compteur
            else:
                no_improve_counter += 1

        best_scores.append(global_best_score)
        print(f"Iteration {iteration+1}/{num_iterations} - Covered Elements: {global_best_covered}, Score: {global_best_score:.4f}")

        # Early Stopping
        if no_improve_counter >= early_stop:
            print(f"⏳ Arrêt anticipé à l'itération {iteration+1} (Pas d'amélioration depuis {early_stop} itérations).")
            break

    execution_time = time.time() - start_time
    print(f"✅ Optimisation terminée en {execution_time:.2f} sec - Meilleure couverture: {global_best_covered}, Score final: {global_best_score:.4f}")
    return global_best, global_best_covered, global_best_score, execution_time

def run_experiments():
    """Exécute l'optimisation PSO sur tous les benchmarks"""
    
    benchmark_folder = "Benchmark"
    benchmark_files = sorted([f for f in os.listdir(benchmark_folder) if f.endswith(".txt")])
    
    results = []

    for file in benchmark_files:
        file_path = os.path.join(benchmark_folder, file)
        universe_size, num_subsets, subsets = read_benchmark(file_path)

        print(f"\n🚀 Exécution de PSO sur {file}...")
        best_solution, best_covered, best_score, exec_time = pso_mcp(subsets, universe_size, num_subsets)

        results.append((file, best_solution, best_covered, best_score, exec_time))

    # Affichage du résumé des résultats
    print("\n📊 Résumé des résultats :")
    for file, best_sol, best_cov, score, time in results:
        print(f"{file}: Nb éléments couverts = {best_cov}, Best Score = {score:.4f}, Time = {time:.2f}s")

if __name__ == "__main__":
    run_experiments()



🚀 Exécution de PSO sur scp41.txt...
Iteration 1/100 - Covered Elements: 929, Score: 4.6450
Iteration 2/100 - Covered Elements: 939, Score: 4.6950
Iteration 3/100 - Covered Elements: 946, Score: 4.7300
Iteration 4/100 - Covered Elements: 948, Score: 4.7400
Iteration 5/100 - Covered Elements: 948, Score: 4.7400
Iteration 6/100 - Covered Elements: 959, Score: 4.7950
Iteration 7/100 - Covered Elements: 959, Score: 4.7950
Iteration 8/100 - Covered Elements: 959, Score: 4.7950
Iteration 9/100 - Covered Elements: 959, Score: 4.7950
Iteration 10/100 - Covered Elements: 959, Score: 4.7950
Iteration 11/100 - Covered Elements: 962, Score: 4.8100
Iteration 12/100 - Covered Elements: 965, Score: 4.8250
Iteration 13/100 - Covered Elements: 967, Score: 4.8350
Iteration 14/100 - Covered Elements: 967, Score: 4.8350
Iteration 15/100 - Covered Elements: 967, Score: 4.8350
Iteration 16/100 - Covered Elements: 967, Score: 4.8350
Iteration 17/100 - Covered Elements: 967, Score: 4.8350
Iteration 18/100 - C